# Compiling Senator and House of Representatives Stock Trading Data


In [7]:
import pandas as pd
import yfinance as yf

## Senate Stocks

In [8]:
senator_trades = pd.read_csv("data/cleaned/all_transactions_senate.csv")
senator_trades = senator_trades.drop("Unnamed: 0", axis=1)
senator_trades.head()

,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper
0,2021-08-18,Joint,SSYS,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,2021,8,18,1001,15000
1,2021-08-12,Joint,SSBK,Purchase,"$50,001 - $100,000",--,Thomas H Tuberville,2021,8,12,50001,100000
2,2021-08-23,Spouse,LMRK,Sale (Full),"$1,001 - $15,000",--,Thomas R Carper,2021,8,23,1001,15000
3,2021-08-11,Spouse,WW,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2021,8,11,1001,15000
4,2021-08-04,Spouse,ARE,Sale (Partial),"$1,001 - $15,000",--,Thomas R Carper,2021,8,4,1001,15000


In [9]:
senate_stocks = yf.download(" ".join(senator_trades["ticker"].unique()),
                            start=senator_trades["transaction_date"].min(),
                            end=senator_trades["transaction_date"].max())
senate_stocks = senate_stocks.drop(["Volume", "Open", "Adj Close", "Low", "High"], axis=1)
senate_stocks

[*********************100%***********************]  909 of 909 completed

121 Failed downloads:
- MCC: No data found, symbol may be delisted
- HF: No data found, symbol may be delisted
- FL4.SG: No data found, symbol may be delisted
- ATML: No data found for this date range, symbol may be delisted
- ACT: Data doesn't exist for startDate = 1347508800, endDate = 1629691200
- VSI: No data found, symbol may be delisted
- WTR: No data found, symbol may be delisted
- CRZO: No data found, symbol may be delisted
- ^RGP: No data found for this date range, symbol may be delisted
- WFM: No data found for this date range, symbol may be delisted
- Q: No data found for this date range, symbol may be delisted
- DWDP: No data found, symbol may be delisted
- CBS: No data found, symbol may be delisted
- IPHI: No data found, symbol may be delisted
- AMSG: No data found for this date range, symbol may be delisted
- CPGX: No data found for this date range, symbol may be delisted
- LLTC: No data found for t

Close                                                       \
           0QZI.IL 3V64.TI           A         AA      AAGIY        AAL   
Date                                                                      
2012-09-13     NaN     NaN   27.453505  23.140890  14.930000  11.250000   
2012-09-14     NaN     NaN   28.419170  23.645519  15.150000  10.530000   
2012-09-17     NaN     NaN   28.347639  23.020741  14.950000  10.560000   
2012-09-18     NaN     NaN   28.290415  22.804470  15.000000  10.310000   
2012-09-19     NaN     NaN   28.526466  22.756411  14.850000  10.970000   
...            ...     ...         ...        ...        ...        ...   
2021-08-17   361.5     NaN  160.910004  41.610001  49.689999  19.340000   
2021-08-18   357.5     NaN  163.020004  41.490002  49.709999  19.139999   
2021-08-19   355.5     NaN  167.669998  36.919998  50.500000  18.730000   
2021-08-20   357.5     NaN  168.130005  37.709999  49.740002  18.500000   
2021-08-23     NaN     NaN         NaN        NaN        NaN        NaN   

                                                         ...                  \
                  AAN       AAON        AAPL        AAT  ...       YUMC ZAYO   
Date                                                     ...                   
2012-09-13        NaN   8.617778   24.392143  27.580000  ...        NaN  NaN   
2012-09-14        NaN   9.088889   24.688572  27.860001  ...        NaN  NaN   
2012-09-17        NaN   8.871111   24.992144  28.000000  ...        NaN  NaN   
2012-09-18        NaN   9.075556   25.068214  27.879999  ...        NaN  NaN   
2012-09-19        NaN   9.000000   25.075001  27.500000  ...        NaN  NaN   
...               ...        ...         ...        ...  ...        ...  ...   
2021-08-17  27.799999  68.070000  150.190002  37.799999  ...  60.279999  NaN   
2021-08-18  28.230000  67.040001  146.360001  37.630001  ...  59.130001  NaN   
2021-08-19  28.299999  68.419998  146.699997  37.570000  ...  58.959999  NaN   
2021-08-20  28.110001  68.940002  148.190002  37.869999  ...  59.169998  NaN   
2021-08-23        NaN        NaN         NaN        NaN  ...        NaN  NaN   

                                                                              
                   ZBH  ZIOP          ZM  ZNGA ZNGA.SW         ZTS ^MWE ^RGP  
Date                                                                          
2012-09-13   66.629997  5.45         NaN  2.96     NaN         NaN  NaN  NaN  
2012-09-14   66.900002  5.53         NaN  3.18     NaN         NaN  NaN  NaN  
2012-09-17   66.930000  5.69         NaN  3.08     NaN         NaN  NaN  NaN  
2012-09-18   67.190002  5.77         NaN  3.08     NaN         NaN  NaN  NaN  
2012-09-19   67.139999  5.71         NaN  3.20     NaN         NaN  NaN  NaN  
...                ...   ...         ...   ...     ...         ...  ...  ...  
2021-08-17  147.860001  1.68  345.540009  8.21     NaN  206.600006  NaN  NaN  
2021-08-18  144.380005  1.66  340.929993  8.20     NaN  203.330002  NaN  NaN  
2021-08-19  144.119995  1.44  332.380005  8.37     NaN  206.440002  NaN  NaN  
2021-08-20  145.460007  1.54  336.859985  8.51     NaN  207.050003  NaN  NaN  
2021-08-23         NaN   NaN         NaN   NaN     NaN         NaN  NaN  NaN  

[2308 rows x 909 columns]

In [10]:
trades = []
for senator in senator_trades["senator"].unique():
    data = senator_trades[senator_trades["senator"] == senator]
    data = data.sort_values("transaction_date", ascending=True)

    temp = {}

    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "Sale (Full)":
            if row.ticker in temp and row.ticker in senate_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "Sale (Partial)":
            if row.ticker in temp:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "Purchase":
            row_dict = {}
            for field in senator_trades.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

senator_trades = pd.DataFrame(trades)

'2020-02-22'
'2020-02-22'
'2019-01-19'


In [ ]:
senator_trades["ticker"].apply(lambda s: yf.Ticker(s).info["sector"])

In [ ]:
senator_trades.to_csv("data/cleaned/senator_trades.csv")

## House of Representatives Stocks

In [ ]:
house_trades = pd.read_csv("data/cleaned/all_transactions_house.csv")
house_trades = house_trades.drop("Unnamed: 0", axis=1)
house_trades

In [ ]:
house_stocks = yf.download(" ".join(house_trades["ticker"].unique()),
                            start=house_trades["transaction_date"].min(),
                            end=house_trades["transaction_date"].max())
house_stocks = house_stocks.drop(["Volume", "Open", "Adj Close", "Low", "High"], axis=1)
house_stocks

In [ ]:
trades = []
for representative in house_trades["representative"].unique():
    data = house_trades[house_trades["representative"] == representative]
    data = data.sort_values("transaction_date", ascending=True)
    temp = {}
    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "sale_full":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "sale_partial":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "purchase":
            row_dict = {}
            for field in house_trades.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

house_trades = pd.DataFrame(trades)

In [ ]:
house_trades.to_csv("data/cleaned/house_trades.csv")

In [ ]:
spy = yf.download("spy",
            min(senator_trades["transaction_date"].min(), house_trades["transaction_date"].min()),
            max(senator_trades["sell_date"].max(), house_trades["sell_date"].max())
            )
spy.to_csv("data/cleaned/spy.csv")